Na aula de hoje vamos trabalhar com um conjunto de dados sobre a resistência compressiva do concreto, que varia em função dos seus ingredientes e da sua idade. Mais informações sobre o conjunto de dados podem ser obtidas [aqui](https://archive.ics.uci.edu/dataset/165/concrete+compressive+strength).

Número de amostras: 1030

Número de atributos: 9

cement (kg/m3)               
furnace_slag (kg/m3)         
fly-ash (kg/m3)               
water (kg/m3)                 
superplasticizer (kg/m3)      
coarse_aggregate (kg/m3)      
fine_aggregate (kg/m3)        
age (days)                  
compressive_strength (MPa) (variável alvo)

In [ ]:
# Clonando pasta do github
if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/tiagofiorini/MLinPhysics.git
    import os as os
    os.chdir('./MLinPhysics')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Análise Exploratória

Todas as variáveis preditora (features) são numéricas. Não há valores faltantes.

Variável alvo: compressive_strength (numérica)

In [ ]:
# Ler arquivo
caminho = '/content/drive/MyDrive/Colab Notebooks/Lu/ML_ifusp/testes/'
df = pd.read_csv(caminho +'Concrete_Data.csv', header = 0, decimal = '.', sep = ",")
df.info()

Há dados faltantes?

In [ ]:
# Há dados faltantes?
# Counting NaN values in all columns
print(df.isna().sum())

**Variáveis numéricas**

Observe que várias features não possuem distribuição normal, com grande presença de valores nulos (variáveis esparsas). Além disso, "age" é uma variável discreta.

Observe também que só a variável "cement" é linearmente correlacionada com a variável alvo.

Os preditores não são fortemente correlacionados, de modo que não há multicolinearidade relevante.

In [ ]:
# Distribuição das variáveis preditoras e alvo
# Boxplots para todas as variáveis
names = df.columns
fig, axes = plt.subplots(3,3)
for name, ax in zip(names, axes.flatten()):
    sns.boxplot(y=name, data=df, notch=True, ax=ax)
plt.tight_layout()

In [ ]:
# Avaliando correlações
sns.pairplot(df)

In [ ]:
sns.heatmap(df.corr(), annot=True, cmap='BrBG')

# Preparação dos dados

*Exercício: avalie mudanças no desempenho dos modelos para diferentes procedimentos de preparação de dados.*
*   *Utilizar outra estratégia de padronização ou normalização (MinMaxScaler, MaxAbsScaler)*
*   *Utilizar validação cruzada k-fold*










In [ ]:
# Padronizando as variáveis
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
scaler = MinMaxScaler()
dfs = scaler.fit_transform(df)
dfs = pd.DataFrame(dfs)
dfs.columns=df.columns.values

In [ ]:
# Histogramas as variáveis escalonadas
names = dfs.columns
fig, axes = plt.subplots(3,3)
for name, ax in zip(names, axes.flatten()):
    sns.histplot(x=name, data=dfs, ax=ax)
plt.tight_layout()

In [ ]:
# Separação de variáveis preditoras e alvo
X = dfs.drop(['compressive_strength'], axis=1) # features
y = dfs.compressive_strength # target

In [ ]:
# Particionamento em treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=18)

# Regressão linear simples

Só a variável "cement" possui alguma correlação linear com a variável alvo. Qual é a fração da variância da resistência compressiva que é explicada pela variável "cement"?

In [ ]:
# Regressão linear simples - sklearn
from sklearn import linear_model

X_train_cement = np.array(X_train['cement']).reshape(-1, 1)
regr = linear_model.LinearRegression() # Create regressor
regr.fit(X_train_cement, y_train) # Train regressor

print('Intercepto:', regr.intercept_)
print('Coeficientes:', regr.coef_)
print('R^2 = ', regr.score(X_train_cement, y_train)) # Coefficient of determination R^2

Avaliando os resíduos

In [ ]:
# Gráfico dos resíduos para o conjunto de treinamento
from sklearn.metrics import PredictionErrorDisplay
y_pred = regr.predict(X_train_cement)
display = PredictionErrorDisplay(y_true=y_train, y_pred=y_pred)
display.plot()
plt.show()

In [ ]:
# Gráfico quantil-quantil (QQ plot)
import statsmodels.api as sm
resid = y_pred-y_train
sm.qqplot(resid.to_numpy(), line='45')

Métricas de desempenho

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
print('R^2 = ', r2_score(y_train, y_pred))
print('MSE = ', mean_squared_error(y_train, y_pred))
print('MAE = ', mean_absolute_error(y_train, y_pred))

# Regressão linear múltipla

*Exercício: Criar um modelo linear com todos os preditores*

**Modelo com todos os preditores**

A biblioteca stasmodels fornece um relatório com diversas métricas de qualidade do ajuste, bem como os coeficientes ajustados, sua margem de erro e significância estatística.

Observe que o intercepto e as variáveis coarse_aggregate e fine_aggregate não foram estatisticamente significativas. Ou seja, sua contribuição para explicar a variância da resistência compressiva não é significativa.

In [ ]:
# Modelo com todos os preditores - statsmodels
import statsmodels.api as sm
X_trainsm = sm.add_constant(X_train) # adicionar uma coluna constante para o statsmodels ajustar um itercepto
mod = sm.OLS(y_train, X_trainsm) # ajustar o modelo
regsm = mod.fit()
print(regsm.summary())

In [ ]:
Exercício: Criar um modelo linear só com os preditores estatisticamente significativos

**Modelo com 6 preditores e sem intercepto**

In [ ]:
# Gráfico dos coeficientes ajustados
#fig = sns.barplot (x=np.arange(1, len(coef)+1), y=coef)
3plt.xlabel("Preditores")
#plt.ylabel("Coeficientes ajustados")

Avaliando os resíduos

In [ ]:
# Gráfico dos resíduos para o conjunto de treinamento
from sklearn.metrics import PredictionErrorDisplay
#display = PredictionErrorDisplay(y_true=y_train, y_pred=y_pred)
#display.plot()
#plt.title('Resíduos - Modelo com 6 preditores')
#plt.show()

Comparando as métricas de desempenho nos conjuntos de treino e teste, podemos avaliar se houve overfitting ou não. Neste caso, as métricas foram melhores no conjunto de teste, o que indica que o modelo tem uma boa capacidade de generalização, sem fazer overfitting.

In [ ]:
# Métricas de desempenho - conjunto de treino
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
#print('Métricas de desempenho (conjunto de treino) - modelo com 6 preditores')
#print('R^2 = ', r2_score(y_train, y_pred_train))
#print('MSE = ', mean_squared_error(y_train, y_pred_train))
#print('MAE = ', mean_absolute_error(y_train, y_pred_train))

In [ ]:
# Métricas de desempenho - conjunto de teste
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
#print('Métricas de desempenho (conjunto de teste) - modelo com 6 preditores')
#print('R^2 = ', r2_score(y_test, y_pred_test))
#print('MSE = ', mean_squared_error(y_test, y_pred_test))
#print('MAE = ', mean_absolute_error(y_test, y_pred_test))

# Regressão Lasso (L1)

Vamos aplicar uma penalidade do tipo L1 com lambda = 0.001. Vemos que o regressor acaba excluindo as mesmas variáveis que não tinham sido significativas (coarse e fine agregate).






*Exercício: Aumentar o valor de lambda (alpha) e avaliar os impactos nos coeficientes e no valor de R^2. Também é possível usar a busca em grade (Grid Search) para obter o valor de lambda que maximiza o R^2 ou alguma outra métrica de interesse.*

In [ ]:
from sklearn.linear_model import Lasso
lassoReg = Lasso(alpha=0.001)
lassoReg.fit(X_train,y_train)
y_pred_train = lassoReg.predict(X_train) # predição para o conjunto de treino
y_pred_test = lassoReg.predict(X_test) # predição para o conjunto de teste

print('Intercepto:', lassoReg.intercept_)
print(X_train.columns.values)
print('Coeficientes:', lassoReg.coef_)
print('R^2 = ', lassoReg.score(X_train,y_train))

In [ ]:
# Gráfico dos coeficientes ajustados
coef = lassoReg.coef_
fig = sns.barplot (x=np.arange(1, len(coef)+1), y=coef)
plt.xlabel("Preditores")
plt.ylabel("Coeficientes ajustados")
plt.title("Coeficientes - Regressão Lasso")

In [ ]:
print('R^2 = ', r2_score(y_train, y_pred_train))
print('MSE = ', mean_squared_error(y_train, y_pred_train))
print('MAE = ', mean_absolute_error(y_train, y_pred_train))

# Regressão Ridge (L2)

In [ ]:
from sklearn.linear_model import Ridge
ridgeReg = Ridge(alpha=10)
ridgeReg.fit(X_train,y_train)

y_pred_train = ridgeReg.predict(X_train) # predição para o conjunto de treino
y_pred_test = ridgeReg.predict(X_test) # predição para o conjunto de teste

print('Intercepto:', ridgeReg.intercept_)
print(X_train.columns.values)
print('Coeficientes:', ridgeReg.coef_)
print('R^2 = ', ridgeReg.score(X_train,y_train))

In [ ]:
# Gráfico dos coeficientes ajustados
coef = ridgeReg.coef_
fig = sns.barplot (x=np.arange(1, len(coef)+1), y=coef)
plt.xlabel("Preditores")
plt.ylabel("Coeficientes ajustados")
plt.title("Coeficientes - Regressão Ridge")

In [ ]:
print('R^2 = ', r2_score(y_train, y_pred_train))
print('MSE = ', mean_squared_error(y_train, y_pred_train))
print('MAE = ', mean_absolute_error(y_train, y_pred_train))